In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
import sys
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
import geopandas as gpd
import topojson as tp
from shapely import geometry
sys.path.append('..')
from modules.transforms import *

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [5]:
url='https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/historial_comunas.csv'

In [6]:
contagiados_comunal=pd.read_csv(url)
contagiados_comunal['Fecha']=pd.to_datetime(contagiados_comunal['Fecha'],errors='coerce').dt.to_pydatetime()

In [7]:
url='https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/utils/comunasChile.csv'

In [8]:
info_comunal=pd.read_csv(url)

In [9]:
info_comunal=info_comunal.rename(columns={'CUT (Código Único Territorial)':'CUT'})

In [18]:
info_comunal['CUT'].astype(str).str[:-3].unique()

array(['15', '1', '2', '3', '4', '5', '6', '7', '16', '8', '9', '14',
       '10', '11', '12', '13'], dtype=object)

In [10]:
for k in ['@ivanMSC']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema @ivanMSC exists, will not be created


In [11]:
contagiados_comunal.set_index('Fecha',inplace=True)

In [12]:
contagiados_comunal['activos']=rolling(contagiados_comunal,numeric_col='Nuevos Confirmados',group_col='Comuna',window=14) 

In [15]:
contagiados_comunal=contagiados_comunal.reset_index()

In [16]:
contagiados_comunal=contagiados_comunal.drop('Region',axis=1)

In [17]:
contagiados_comunal=contagiados_comunal.merge(info_comunal[['CUT','Provincia','Region','Poblacion','Lat','Lon']],
                         on='CUT',how='left')

In [18]:
contagiados_comunal['Indicencia Acumulados']=contagiados_comunal['Confirmados Acumulados']/contagiados_comunal['Poblacion']*100000

In [19]:
contagiados_comunal['Indicencia Activos']=contagiados_comunal['activos']/contagiados_comunal['Poblacion']*100000

In [20]:
name='contagios_comunal@ivanMSC'
schema='@ivanMSC'
print("creating table "+name+' ,schema: '+schema)
contagiados_comunal.to_sql(name, schema=schema,con=cnx,if_exists='replace')
print("saving table"+path+name+timestamp+'.csv in cache')
contagiados_comunal.to_csv(path+name+timestamp+'.csv',encoding='utf-8')
name='info_comunal@ivanMSC'
print("creating table "+name+' ,schema: '+schema)
info_comunal.to_sql(name, schema=schema,con=cnx,if_exists='replace')
print("saving table"+path+name+timestamp+'.csv in cache')
info_comunal.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

creating table contagios_comunal@ivanMSC ,schema: @ivanMSC
saving table/data/ETLcache/contagios_comunal@ivanMSC_18092020_061032.csv in cache
creating table info_comunal@ivanMSC ,schema: @ivanMSC
saving table/data/ETLcache/info_comunal@ivanMSC_18092020_061032.csv in cache


In [21]:
contagiados_comunal

,Fecha,CUT,Comuna,Confirmados Acumulados,Nuevos Confirmados,activos,Provincia,Region,Poblacion,Lat,Lon,Indicencia Acumulados,Indicencia Activos
0,2020-03-18,15101,Arica,1,1,1.0,Arica,Arica y Parinacota,247552.0,-18.455,-70.290,0.403956,0.403956
1,2020-03-19,15101,Arica,1,0,1.0,Arica,Arica y Parinacota,247552.0,-18.455,-70.290,0.403956,0.403956
2,2020-03-20,15101,Arica,1,0,1.0,Arica,Arica y Parinacota,247552.0,-18.455,-70.290,0.403956,0.403956
3,2020-03-21,15101,Arica,1,0,1.0,Arica,Arica y Parinacota,247552.0,-18.455,-70.290,0.403956,0.403956
4,2020-03-22,15101,Arica,2,1,2.0,Arica,Arica y Parinacota,247552.0,-18.455,-70.290,0.807911,0.807911
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47108,2020-09-13,12301,Porvenir,128,19,66.0,Tierra del Fuego,Magallanes,7323.0,-53.298,-70.385,1747.917520,901.269971
47109,2020-09-14,12301,Porvenir,128,0,65.0,Tierra del Fuego,Magallanes,7323.0,-53.298,-70.385,1747.917520,887.614366
47110,2020-09-15,12301,Porvenir,128,0,65.0,Tierra del Fuego,Magallanes,7323.0,-53.298,-70.385,1747.917520,887.614366
47111,2020-09-16,12301,Porvenir,135,7,72.0,Tierra del Fuego,Magallanes,7323.0,-53.298,-70.385,1843.506760,983.203605
